# 13장  과적합 피하기


### [실습]  초음파 광물 예측

### 1 |  데이터의 확인과 실행
### 2 |  과적합 이해하기
### 3 |  학습셋과 테스트셋
### 4 |  모델 저장과 재사용
### 5 |  k겹 교차 검증


---

실습 데이터 초음파 광물 예측 : dataset/sonar.csv
- https://github.com/gilbutITbook/006958
- https://raw.githubusercontent.com/gilbutITbook/006958/master/deeplearning/dataset/sonar.csv

![광물](13_01.jpg)

In [1]:
!wget --no-check-certificate  https://raw.githubusercontent.com/gilbutITbook/006958/master/deeplearning/dataset/sonar.csv

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2018-08-16 21:40:19--  https://raw.githubusercontent.com/gilbutITbook/006958/master/deeplearning/dataset/sonar.csv
Resolving raw.githubusercontent.com... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com|151.101.0.133|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 87776 (86K) [text/plain]
Saving to: `sonar.csv'

     0K .......... .......... .......... .......... .......... 58%  619K 0s
    50K .......... .......... .......... .....                100% 4.05M=0.09s

2018-08-16 21:40:20 (959 KB/s) - `sonar.csv' saved [87776/87776]



In [2]:
! dir 

 D 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 9495-1634

 D:\Work_Git\DeepMenia\part04\week3_180817 디렉터리

2018-08-16  오후 09:40    <DIR>          .
2018-08-16  오후 09:40    <DIR>          ..
2018-08-16  오후 09:30    <DIR>          .ipynb_checkpoints
2018-08-15  오후 07:32            52,481 01.jpg
2018-08-15  오후 07:35            31,866 02.jpg
2018-08-15  오후 07:38            18,202 03.jpg
2018-08-15  오후 08:09            17,909 04.jpg
2018-08-16  오후 09:39           133,795 12장_다중분류문제해결하기.ipynb
2018-08-16  오후 09:31            99,355 13_01.jpg
2018-08-16  오후 09:40             2,757 13장_과적합피하기.ipynb
2018-08-15  오후 07:27             4,551 iris.csv
2018-08-16  오후 09:40            87,776 sonar.csv
               9개 파일             448,692 바이트
               3개 디렉터리  27,943,014,400 바이트 남음


- 1988년 존스홉킨스대학교의 세즈노프스키(Sejnowski) 교수는 2년 전 힌튼 교수가 발표한 역전파 알고리즘에 관심을 가지고 있었음
- 그는 은닉층과 역전파가 얼마나 큰 효과가 있는지를 직접 실험해 보고 싶었음
- 그래서 광석과 일반 돌을 갖다 놓고 음파 탐지기를 쏜 후 그 결과를 데이터로 정리
- 오차 역전파 알고리즘을 사용한 신경망이 과연 얼마나 광석과 돌을 구분하는데 효과적인지 알아보기 위해서임
- 이때 세즈노프스키 교수가 구현해 낸 신경망의 예측 정확도는 몇 퍼센트(%)였을까?
- 세즈노프스키 교수가 만들어 공개한 음파 탐지기 데이터 정보를 이용해 광물과 돌을 구분하는 실험을 해보자
- 그때나 지금이나 쉽게 해결하지 못하는, 하지만 머신러닝을 하는 사람이라면 반드시 극복해야 하는 문제인 ‘과적합(overfitting)’에 관한 내용도 다루어 보자


---

## 1. 데이터의 확인과 실행

먼저 데이터를 확인해 보자
Dataset/sonar.csv 파일을 가져옴

In [3]:
import pandas as pd

df = pd.read_csv('sonar.csv', header=None)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
0     208 non-null float64
1     208 non-null float64
2     208 non-null float64
3     208 non-null float64
4     208 non-null float64
5     208 non-null float64
6     208 non-null float64
7     208 non-null float64
8     208 non-null float64
9     208 non-null float64
10    208 non-null float64
11    208 non-null float64
12    208 non-null float64
13    208 non-null float64
14    208 non-null float64
15    208 non-null float64
16    208 non-null float64
17    208 non-null float64
18    208 non-null float64
19    208 non-null float64
20    208 non-null float64
21    208 non-null float64
22    208 non-null float64
23    208 non-null float64
24    208 non-null float64
25    208 non-null float64
26    208 non-null float64
27    208 non-null float64
28    208 non-null float64
29    208 non-null float64
30    208 non-null float64
31    208 non-null float64
32    208 non-null float64
33  

![광물](13_02.jpg)

- Index가 208개이므로 총 샘플의 수는 208개이고, 컬럼 수가 61개이므로 60개의 속성과 1개의 클래스로 이루어져 있음을 짐작할 수 있음
- 모든 컬럼이 실수형(float64)인데, 맨 마지막 컬럼만 객체형인 것으로 보아 마지막에 나오는 컬럼은 클래스이며 데이터형 변환이 필요한 것을 알 수 있음
- 실제로 맞는지 일부를 출력해 확인해 보자


In [5]:
print(df.head())

       0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9  ...      51      52      53      54      55      56      57      58  \
0  0.2111 ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084  0.0090   
1  0.2872 ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049  0.0052   
2  0.6194 ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164  0.0095   
3  0.1264 ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044  0.0040   
4  0.4459 ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048  0.0107   

       59  60  
0  0.0032   R  
1  0.0044   R  
2 

위의 정보를 토대로 이제 다음과 같이 딥러닝을 실행해 보자

- 코드 13-1 초음파 광물 예측하기: 데이터 확인과 실행

In [6]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy
import tensorflow as tf

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)

# 데이터 입력
df = pd.read_csv('sonar.csv', header=None)

dataset = df.values
X = dataset[:,0:60]
Y_obj = dataset[:,60]

# 문자열 변환
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

# 모델 설정
model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일 
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

# 모델 실행
model.fit(X, Y, epochs=200, batch_size=5)

# 결과 출력 
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Epoch 1/200
208/208 [==============================] - 0s 2ms/step - loss: 0.2532 - acc: 0.4471
Epoch 2/200
208/208 [==============================] - 0s 312us/step - loss: 0.2457 - acc: 0.5385
Epoch 3/200
208/208 [==============================] - 0s 316us/step - loss: 0.2428 - acc: 0.5385
Epoch 4/200
208/208 [==============================] - 0s 264us/step - loss: 0.2402 - acc: 0.5385
Epoch 5/200
208/208 [==============================] - 0s 332us/step - loss: 0.2349 - acc: 0.5433
Epoch 6/200
208/208 [==============================] - 0s 355us/step - loss: 0.2288 - acc: 0.5529
Epoch 7/200
208/208 [==============================] - 0s 374us/step - loss: 0.2225 - acc: 0.6106
Epoch 8/200
208/208 [==============================] - 0s 252us/step - loss: 0.2165 - acc: 0.6394
Epoch 9/200
208/208 [==============================] - 0s 329us/step - loss: 0.2032 - acc: 0.7019
Epoch 10/200
208/208 [==============================] - 0s 278us/step - loss: 0.1918 - acc: 0.7260
Epoch 11/200
208/208 

208/208 [==============================] - 0s 225us/step - loss: 0.0662 - acc: 0.9231
Epoch 84/200
208/208 [==============================] - 0s 228us/step - loss: 0.0672 - acc: 0.9279
Epoch 85/200
208/208 [==============================] - 0s 206us/step - loss: 0.0660 - acc: 0.9375
Epoch 86/200
208/208 [==============================] - 0s 222us/step - loss: 0.0640 - acc: 0.9423
Epoch 87/200
208/208 [==============================] - 0s 211us/step - loss: 0.0633 - acc: 0.9423
Epoch 88/200
208/208 [==============================] - 0s 216us/step - loss: 0.0621 - acc: 0.9519
Epoch 89/200
208/208 [==============================] - 0s 211us/step - loss: 0.0618 - acc: 0.9327
Epoch 90/200
208/208 [==============================] - 0s 222us/step - loss: 0.0623 - acc: 0.9231
Epoch 91/200
208/208 [==============================] - 0s 220us/step - loss: 0.0608 - acc: 0.9519
Epoch 92/200
208/208 [==============================] - 0s 230us/step - loss: 0.0639 - acc: 0.9135
Epoch 93/200
208/208 [=

208/208 [==============================] - 0s 292us/step - loss: 0.0204 - acc: 0.9856
Epoch 166/200
208/208 [==============================] - 0s 296us/step - loss: 0.0214 - acc: 0.9808
Epoch 167/200
208/208 [==============================] - 0s 303us/step - loss: 0.0210 - acc: 0.9856
Epoch 168/200
208/208 [==============================] - 0s 396us/step - loss: 0.0226 - acc: 0.9856
Epoch 169/200
208/208 [==============================] - 0s 326us/step - loss: 0.0201 - acc: 0.9856
Epoch 170/200
208/208 [==============================] - 0s 302us/step - loss: 0.0202 - acc: 0.9856
Epoch 171/200
208/208 [==============================] - 0s 288us/step - loss: 0.0197 - acc: 0.9856
Epoch 172/200
208/208 [==============================] - 0s 321us/step - loss: 0.0193 - acc: 0.9856
Epoch 173/200
208/208 [==============================] - 0s 299us/step - loss: 0.0196 - acc: 0.9856
Epoch 174/200
208/208 [==============================] - 0s 320us/step - loss: 0.0188 - acc: 0.9856
Epoch 175/200


----

## 2. 과적합 이해하기

- 이제 과적합 문제가 무엇인지 알아보고 이를 어떻게 해결하는지 살펴보자
- 과적합(over fitting)이란 모델이 학습 데이터셋 안에서는 일정 수준 이상의 예측 정확도를 보이지만, 새로운 데이터에 적용하면 잘 맞지 않는 것을 말함
- 그림 13-1의 그래프에서 빨간색을 보면 주어진 샘플에 정확히 맞게끔 선이 그어져 있음
- 하지만 이 선은 너무 이 경우에만 최적화되어 있음
- 다시 말해서, 완전히 새로운 데이터에 적용하면 이 선을 통해 정확히 두 그룹을 나누지 못하게 된다는 의미

![광물](13_03.jpg)

- 과적합은 층이 너무 많거나 변수가 복잡해서 발생하기도 하고 테스트셋과 학습셋이 중복될 때 생기기도 함
- 특히 딥러닝은 학습 단계에서 입력, 은닉층, 출력층의 노드들에 상당히 많은 변수들이 투입됨
- 따라서 딥러닝을 진행하는 동안 과적합에 빠지지 않게 늘 주의해야 함

---

## 3. 학습셋과 데이터셋

- 그렇다면 과적합을 방지하려면 어떻게 해야 할까? 
- 먼저 학습을 하는 데이터셋과 이를 테스트할 데이터셋을 완전히 구분한 다음 학습과 동시에 테스트를 병행하며 진행하는 것이 한 방법
- 예를 들어, 데이터셋이 총 100개의 샘플로 이루어져 있다면 다음과 같이 두 개의 셋으로 나눔

![광물](13_04.jpg)

- 신경망을 만들어 70개의 샘플로 학습을 진행한 후 이 학습의 결과를 저장
- 이렇게 저장된 파일을 ‘모델’이라고 부름
- 모델은 다른 셋에 적용할 경우 학습 단계에서 각인되었던 그대로 다시 수행함
- 따라서 나머지 30개의 샘플로 실험해서 정확도를 살펴보면 학습이 얼마나 잘 되었는지를 알 수 있는 것
- 딥러닝 같은 알고리즘을 충분히 조절하여 가장 나은 모델이 만들어지면, 이를 실생활에 대입하여 활용하는 것이 바로 머신러닝의 개발 순서

![광물](13_05.jpg)

- 지금까지 우리는 테스트셋을 만들지 않고 학습해 왔음
- 그런데도 매번 우리는 정확도(Accuracy)를 계산할 수 있었음
- 지금까지 학습 데이터를 이용해 정확도를 측정한 것은 데이터에 들어있는 모든 샘플을 그대로 테스트에 활용한 결과
- 앞서 설명한 대로 학습에 사용된 샘플은 테스트에 쓸 수 없으므로 학습 단계에서 테스트할 샘플은 자동으로 빼고, 이를 테스트한 결과를 모아 정확도를 계산하는 것
- 이러한 방법은 빠른 시간에 모델 성능을 파악하고 수정할 수 있도록 도와줌
- 하지만 머신러닝의 최종 목적은 과거의 데이터를 토대로 새로운 데이터를 예측하는 것
- 즉, 새로운 데이터에 사용할 모델을 만드는 것이 최종 목적이므로 테스트셋을 만들어 정확한 평가를 병행하는 것이 매우 중요함


- 학습셋만 가지고 평가할때, 층을 더하거나 에포크(epoch) 값을 높여 실행 횟수를 늘리면 정확도가 계속해서 올라갈 수 있음
- 하지만 학습 데이터셋만으로 평가한 예측 성공률이 테스트셋에서도 그대로 나타나지는 않음
- 학습이 깊어져서 학습셋 내부에서의 성공률은 높아져도 테스트셋에서는 효과가 없다면 과적합이 일어나고 있는 것
- 이를 그래프로 표현하면 그림 13-3과 같음

![광물](13_06.jpg)
- 그림 13-3 학습이 계속되면 학습셋에서의 정확도는 계속 올라가지만, 테스트셋에서는 과적합이 발생!


- 학습을 진행해도 테스트 결과가 더 이상 좋아지지 않는 지점에서 학습을 멈춰야 함
- 이때의 학습 정도가 가장 적절한 것으로 볼 수 있음


- 우리가 다루는 초음파 광물 예측 데이터를 만든 세즈노프스키 교수가 실험 결과를 발표한 논문의 일부를 가져와 보자

![광물](13_07.jpg)

- 그림 13-4 학습셋과 테스트셋 정확도 측정의 예(RP Gorman et.al., 1998)


- 여기서 눈여겨 봐야 할 부분은 은닉층(Number of Hidden Units) 수가 올라감에 따라 학습셋의 예측률(Average Performance on Training Sets)과 테스트셋의 예측률(Average Performance on Testing Sets)이 어떻게 변하는지임
- 은닉층이 늘어날수록 학습셋의 예측률이 점점 올라가다가 결국 24개 층에 이르면 100%의 예측률을 보임
- 하지만 이 모델을 토대로 테스트한 결과는 어떤가? 
- 테스트셋 예측률은 12개에서 90.4%로 최고를 이루다 24개째에선 다시 89.2%로 떨어지고 맒
- 즉, 식이 복잡해지고 학습량이 늘어날수록 학습 데이터를 통한 예측률은 계속해서 올라가지만, 테스트셋을 이용한 예측률은 오히려 떨어짐


- 예제에 주어진 데이터를 학습셋과 테스트셋으로 나누는 예제를 만들어 보자
- 불러온 X 데이터와 Y 데이터에서 각각 정해진 비율(%)만큼 구분하여 한 그룹은 학습에 사용하고 다른 한 그룹은 테스트에 사용하게 하는 함수가 sklearn 라이브러리의 train_test_split() 함수
- 따라서 다음과 같이 학습셋과 테스트셋을 만들 수 있음
- 다음은 학습셋을 70%, 테스트셋을 30%로 설정했을 때의 예

> from sklearn.model_selection import train_test_split

> \# 학습셋과 테스트셋의 구분

> X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)


- 모델을 실행하는 부분에서 위에서 만들어진 학습셋으로 학습을, 테스트셋으로 테스트를 하게 하려면 다음과 같이 실행


> model.fit(X_train, Y_train, epochs=130, batch_size=5)

> \# 테스트셋에 모델 적용 

> print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test) [1]))


코드 13-2 초음파 광물 예측하기: 학습셋과 테스트셋 구분

In [7]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy
import tensorflow as tf

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)
df = pd.read_csv('sonar.csv', header=None)


In [8]:
dataset = df.values
X = dataset[:,0:60]
Y_obj = dataset[:,60]

e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

# 학습셋과 테스트셋의 구분
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [9]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=130, batch_size=5)

# 테스트셋에 모델 적용 
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test) [1]))

Epoch 1/130
145/145 [==============================] - 0s 2ms/step - loss: 0.2522 - acc: 0.5310
Epoch 2/130
145/145 [==============================] - 0s 431us/step - loss: 0.2462 - acc: 0.5172
Epoch 3/130
145/145 [==============================] - 0s 477us/step - loss: 0.2440 - acc: 0.5379
Epoch 4/130
145/145 [==============================] - 0s 473us/step - loss: 0.2417 - acc: 0.5310
Epoch 5/130
145/145 [==============================] - 0s 289us/step - loss: 0.2395 - acc: 0.5379
Epoch 6/130
145/145 [==============================] - 0s 287us/step - loss: 0.2377 - acc: 0.5379
Epoch 7/130
145/145 [==============================] - 0s 330us/step - loss: 0.2335 - acc: 0.5379
Epoch 8/130
145/145 [==============================] - 0s 296us/step - loss: 0.2303 - acc: 0.5448
Epoch 9/130
145/145 [==============================] - 0s 306us/step - loss: 0.2223 - acc: 0.6000
Epoch 10/130
145/145 [==============================] - 0s 310us/step - loss: 0.2150 - acc: 0.6414
Epoch 11/130
145/145 

145/145 [==============================] - 0s 269us/step - loss: 0.0614 - acc: 0.9379
Epoch 84/130
145/145 [==============================] - 0s 297us/step - loss: 0.0596 - acc: 0.9379
Epoch 85/130
145/145 [==============================] - 0s 298us/step - loss: 0.0595 - acc: 0.9448
Epoch 86/130
145/145 [==============================] - 0s 296us/step - loss: 0.0573 - acc: 0.9448
Epoch 87/130
145/145 [==============================] - 0s 268us/step - loss: 0.0580 - acc: 0.9310
Epoch 88/130
145/145 [==============================] - 0s 256us/step - loss: 0.0559 - acc: 0.9379
Epoch 89/130
145/145 [==============================] - 0s 296us/step - loss: 0.0518 - acc: 0.9448
Epoch 90/130
145/145 [==============================] - 0s 303us/step - loss: 0.0517 - acc: 0.9586
Epoch 91/130
145/145 [==============================] - 0s 254us/step - loss: 0.0586 - acc: 0.9103
Epoch 92/130
145/145 [==============================] - 0s 263us/step - loss: 0.0523 - acc: 0.9586
Epoch 93/130
145/145 [=

- 테스트셋으로 실험해 본 결과 80.95%의 예측 성공률을 보임
- 앞서(코드 13-1) 학습셋만으로 실험했을 때의 99.52%와 비교해 보면 차이가 남


---

## 4. 모델 저장과 재사용

- 학습이 끝난 후 테스트해 본 결과가 만족스러울 때 이를 모델로 저장하여 새로운 데이터에 사용할 수 있음
- 앞서 학습한 결과를 모델로 저장하려면 다음과 같이 실행

> from keras.models import load_model

> model.save('my_model.h5')

- 이를 불러오려면 다음과 같이 실행

> model = load_model ('my_model.h5')


- 코드 13-3 초음파 광물 예측하기: 모델 저장과 재사용


In [10]:
from keras.models import Sequential, load_model
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy
import tensorflow as tf

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)

df = pd.read_csv('sonar.csv', header=None)

In [11]:
dataset = df.values
X = dataset[:,0:60]
Y_obj = dataset[:,60]

e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)
# 학습셋과 테스트셋을 나눔
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=130, batch_size=5)
model.save('my_model.h5')   # 모델을 컴퓨터에 저장

del model       # 테스트를 위해 메모리 내의 모델을 삭제
model = load_model('my_model.h5') # 모델을 새로 불러옴

print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))  # 불러온 모델로 테스트 실행

Epoch 1/130
145/145 [==============================] - 0s 2ms/step - loss: 0.2522 - acc: 0.5310
Epoch 2/130
145/145 [==============================] - 0s 246us/step - loss: 0.2462 - acc: 0.5172
Epoch 3/130
145/145 [==============================] - 0s 426us/step - loss: 0.2440 - acc: 0.5379
Epoch 4/130
145/145 [==============================] - 0s 239us/step - loss: 0.2417 - acc: 0.5310
Epoch 5/130
145/145 [==============================] - 0s 344us/step - loss: 0.2398 - acc: 0.5379
Epoch 6/130
145/145 [==============================] - 0s 248us/step - loss: 0.2373 - acc: 0.5379
Epoch 7/130
145/145 [==============================] - 0s 420us/step - loss: 0.2339 - acc: 0.5379
Epoch 8/130
145/145 [==============================] - 0s 254us/step - loss: 0.2305 - acc: 0.5448
Epoch 9/130
145/145 [==============================] - 0s 385us/step - loss: 0.2226 - acc: 0.6069
Epoch 10/130
145/145 [==============================] - 0s 270us/step - loss: 0.2151 - acc: 0.6345
Epoch 11/130
145/145 

145/145 [==============================] - 0s 273us/step - loss: 0.0674 - acc: 0.9310
Epoch 84/130
145/145 [==============================] - 0s 275us/step - loss: 0.0642 - acc: 0.9241
Epoch 85/130
145/145 [==============================] - 0s 298us/step - loss: 0.0641 - acc: 0.9310
Epoch 86/130
145/145 [==============================] - 0s 241us/step - loss: 0.0631 - acc: 0.9172
Epoch 87/130
145/145 [==============================] - 0s 263us/step - loss: 0.0626 - acc: 0.9310
Epoch 88/130
145/145 [==============================] - 0s 262us/step - loss: 0.0610 - acc: 0.9241
Epoch 89/130
145/145 [==============================] - 0s 300us/step - loss: 0.0572 - acc: 0.9379
Epoch 90/130
145/145 [==============================] - 0s 247us/step - loss: 0.0573 - acc: 0.9448
Epoch 91/130
145/145 [==============================] - 0s 248us/step - loss: 0.0652 - acc: 0.8966
Epoch 92/130
145/145 [==============================] - 0s 241us/step - loss: 0.0566 - acc: 0.9517
Epoch 93/130
145/145 [=

---

## 5. k겹 교차 검증

- 테스트를 더 정확하게 설정할수록 세상으로 나왔을 때 더 잘 작동한다고 할 수 있음
- 하지만 문제는 데이터가 그만큼 충분하지 않을 때 발생
- 딥러닝 혹은 머신러닝 작업을 할 때 늘 어려운 문제 중 하나는 알고리즘을 충분히 테스트하였어도 데이터가 충분치 않으면 좋은 결과를 내기가 어렵다는 것 
- 앞서 가지고 있는 데이터의 약 70%를 학습셋으로 써야 했으므로 테스트셋은 겨우 전체 데이터의 30%에 그쳤음
- 이 정도 테스트만으로는 실제로 얼마나 잘 작동하는지 확신하기는 쉽지 않음
- 이러한 단점을 보완하고자 만든 방법이 바로 k겹 교차 검증(k-fold cross validation)
- k겹 교차 검증이란 데이터셋을 여러 개로 나누어 하나씩 테스트셋으로 사용하고 나머지를 모두 합해서 학습셋으로 사용하는 방법
- 이렇게 하면 가지고 있는 데이터의 100%를 테스트셋으로 사용할 수 있음
- 예를 들어, 5겹 교차 검증(5-fold cross validation)의 예가 그림 13-5에 설명되어 있음

![광물](13_08.jpg)

- 이제 k겹 교차 검증을 실시해 보자
- 데이터를 원하는 숫자만큼 쪼개 각각 학습셋과 테스트셋으로 사용되게 만드는 함수는 sklearn의 StratifiedKFold()

> from sklearn.model_selection import StratifiedKFold

> n_fold = 10

> skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)


- 10개의 파일로 쪼개 테스트하는 10-fold cross validation을 실시하도록 n_ fold의 값을 10으로 설정한 뒤 StratifiedKFold() 함수에 적용했음
- 그런 다음 모델을 만들고 실행하는 부분을 for 구문으로 묶어 n_fold만큼 반복되게 함 


> for train, test in skf.split(X, Y):

>     model = Sequential()

>     model.add(Dense(24, input_dim=60, activation='relu'))

>     model.add(Dense(10, activation='relu'))

>     model.add(Dense(1, activation='sigmoid'))

>     model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

>     model.fit(X[train], Y[train], epochs=100, batch_size=5)


- 정확도(Accuracy)를 매번 저장하여 한 번에 보여줄 수 있게 accuracy 배열을 만들어 보자

> accuracy = []

> for train, test in skf.split(X, Y):

> (생략)

>     k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])

>     accuracy.append(k_accuracy)

> print("\n %.f fold accuracy:" % n_fold, accuracy)


- 이를 종합하여 10-fold cross validation이 포함된 스크립트를 다음과 같이 완성할 수 있음


코드 13-4 초음파 광물 예측하기: k겹 교차 검증

In [13]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import numpy
import pandas as pd
import tensorflow as tf

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)


In [14]:
df = pd.read_csv('sonar.csv', header=None)

dataset = df.values
X = dataset[:,0:60]
Y_obj = dataset[:,60]

e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

# 10개의 파일로 쪼갬
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

# 빈 accuracy 배열
accuracy = []


In [15]:
# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

# 결과 출력 
print("\n %.f fold accuracy:" % n_fold, accuracy)


Epoch 1/100
186/186 [==============================] - 0s 2ms/step - loss: 0.2520 - acc: 0.4946
Epoch 2/100
186/186 [==============================] - 0s 317us/step - loss: 0.2450 - acc: 0.5699
Epoch 3/100
186/186 [==============================] - 0s 282us/step - loss: 0.2418 - acc: 0.5323
Epoch 4/100
186/186 [==============================] - 0s 359us/step - loss: 0.2388 - acc: 0.5376
Epoch 5/100
186/186 [==============================] - 0s 391us/step - loss: 0.2337 - acc: 0.5430
Epoch 6/100
186/186 [==============================] - 0s 445us/step - loss: 0.2239 - acc: 0.6022
Epoch 7/100
186/186 [==============================] - 0s 268us/step - loss: 0.2134 - acc: 0.6290
Epoch 8/100
186/186 [==============================] - 0s 328us/step - loss: 0.2011 - acc: 0.7043
Epoch 9/100
186/186 [==============================] - 0s 284us/step - loss: 0.1892 - acc: 0.7151
Epoch 10/100
186/186 [==============================] - 0s 338us/step - loss: 0.1796 - acc: 0.7796
Epoch 11/100
186/186 

186/186 [==============================] - 0s 303us/step - loss: 0.0404 - acc: 0.9624
Epoch 84/100
186/186 [==============================] - 0s 322us/step - loss: 0.0420 - acc: 0.9677
Epoch 85/100
186/186 [==============================] - 0s 280us/step - loss: 0.0497 - acc: 0.9516
Epoch 86/100
186/186 [==============================] - 0s 241us/step - loss: 0.0406 - acc: 0.9462
Epoch 87/100
186/186 [==============================] - 0s 236us/step - loss: 0.0389 - acc: 0.9677
Epoch 88/100
186/186 [==============================] - 0s 236us/step - loss: 0.0358 - acc: 0.9731
Epoch 89/100
186/186 [==============================] - 0s 260us/step - loss: 0.0357 - acc: 0.9731
Epoch 90/100
186/186 [==============================] - 0s 301us/step - loss: 0.0357 - acc: 0.9677
Epoch 91/100
186/186 [==============================] - 0s 306us/step - loss: 0.0354 - acc: 0.9624
Epoch 92/100
186/186 [==============================] - 0s 251us/step - loss: 0.0364 - acc: 0.9624
Epoch 93/100
186/186 [=

187/187 [==============================] - 0s 368us/step - loss: 0.0670 - acc: 0.9144
Epoch 66/100
187/187 [==============================] - 0s 352us/step - loss: 0.0672 - acc: 0.9251
Epoch 67/100
187/187 [==============================] - 0s 400us/step - loss: 0.0647 - acc: 0.9198
Epoch 68/100
187/187 [==============================] - 0s 261us/step - loss: 0.0648 - acc: 0.9144
Epoch 69/100
187/187 [==============================] - 0s 240us/step - loss: 0.0630 - acc: 0.9251
Epoch 70/100
187/187 [==============================] - 0s 331us/step - loss: 0.0612 - acc: 0.9198
Epoch 71/100
187/187 [==============================] - 0s 409us/step - loss: 0.0645 - acc: 0.9091
Epoch 72/100
187/187 [==============================] - 0s 303us/step - loss: 0.0674 - acc: 0.9037
Epoch 73/100
187/187 [==============================] - 0s 283us/step - loss: 0.0608 - acc: 0.9358
Epoch 74/100
187/187 [==============================] - 0s 256us/step - loss: 0.0588 - acc: 0.9305
Epoch 75/100
187/187 [=

187/187 [==============================] - 0s 336us/step - loss: 0.0876 - acc: 0.8877
Epoch 48/100
187/187 [==============================] - 0s 283us/step - loss: 0.0844 - acc: 0.8877
Epoch 49/100
187/187 [==============================] - 0s 240us/step - loss: 0.0809 - acc: 0.8930
Epoch 50/100
187/187 [==============================] - 0s 246us/step - loss: 0.0778 - acc: 0.8930
Epoch 51/100
187/187 [==============================] - 0s 245us/step - loss: 0.0789 - acc: 0.8984
Epoch 52/100
187/187 [==============================] - 0s 240us/step - loss: 0.0747 - acc: 0.9305
Epoch 53/100
187/187 [==============================] - 0s 247us/step - loss: 0.0733 - acc: 0.9251
Epoch 54/100
187/187 [==============================] - 0s 282us/step - loss: 0.0716 - acc: 0.9144
Epoch 55/100
187/187 [==============================] - 0s 256us/step - loss: 0.0717 - acc: 0.9091
Epoch 56/100
187/187 [==============================] - 0s 259us/step - loss: 0.0782 - acc: 0.8877
Epoch 57/100
187/187 [=

187/187 [==============================] - 0s 324us/step - loss: 0.0928 - acc: 0.8930
Epoch 30/100
187/187 [==============================] - 0s 321us/step - loss: 0.0939 - acc: 0.8770
Epoch 31/100
187/187 [==============================] - 0s 272us/step - loss: 0.0901 - acc: 0.9198
Epoch 32/100
187/187 [==============================] - 0s 251us/step - loss: 0.0878 - acc: 0.9091
Epoch 33/100
187/187 [==============================] - 0s 283us/step - loss: 0.0854 - acc: 0.9091
Epoch 34/100
187/187 [==============================] - 0s 251us/step - loss: 0.0895 - acc: 0.8984
Epoch 35/100
187/187 [==============================] - 0s 253us/step - loss: 0.0829 - acc: 0.9037
Epoch 36/100
187/187 [==============================] - 0s 261us/step - loss: 0.0814 - acc: 0.9091
Epoch 37/100
187/187 [==============================] - 0s 261us/step - loss: 0.0782 - acc: 0.9198
Epoch 38/100
187/187 [==============================] - 0s 279us/step - loss: 0.0764 - acc: 0.9305
Epoch 39/100
187/187 [=

...

187/187 [==============================] - 0s 325us/step - loss: 0.0729 - acc: 0.9198
Epoch 58/100
187/187 [==============================] - 0s 330us/step - loss: 0.0706 - acc: 0.9358
Epoch 59/100
187/187 [==============================] - 0s 380us/step - loss: 0.0712 - acc: 0.9251
Epoch 60/100
187/187 [==============================] - 0s 325us/step - loss: 0.0693 - acc: 0.9358
Epoch 61/100
187/187 [==============================] - 0s 350us/step - loss: 0.0690 - acc: 0.9305
Epoch 62/100
187/187 [==============================] - 0s 350us/step - loss: 0.0711 - acc: 0.9198
Epoch 63/100
187/187 [==============================] - 0s 336us/step - loss: 0.0665 - acc: 0.9358
Epoch 64/100
187/187 [==============================] - 0s 334us/step - loss: 0.0636 - acc: 0.9412
Epoch 65/100
187/187 [==============================] - 0s 304us/step - loss: 0.0658 - acc: 0.9091
Epoch 66/100
187/187 [==============================] - 0s 344us/step - loss: 0.0629 - acc: 0.9358
Epoch 67/100
187/187 [=

188/188 [==============================] - 0s 440us/step - loss: 0.0891 - acc: 0.8989
Epoch 40/100
188/188 [==============================] - 0s 430us/step - loss: 0.0795 - acc: 0.9096
Epoch 41/100
188/188 [==============================] - 0s 356us/step - loss: 0.0786 - acc: 0.9202
Epoch 42/100
188/188 [==============================] - 0s 297us/step - loss: 0.0786 - acc: 0.9096
Epoch 43/100
188/188 [==============================] - 0s 286us/step - loss: 0.0779 - acc: 0.9202
Epoch 44/100
188/188 [==============================] - 0s 281us/step - loss: 0.0737 - acc: 0.9362
Epoch 45/100
188/188 [==============================] - 0s 271us/step - loss: 0.0730 - acc: 0.9255
Epoch 46/100
188/188 [==============================] - 0s 303us/step - loss: 0.0703 - acc: 0.9309
Epoch 47/100
188/188 [==============================] - 0s 408us/step - loss: 0.0722 - acc: 0.9309
Epoch 48/100
188/188 [==============================] - 0s 331us/step - loss: 0.0768 - acc: 0.9202
Epoch 49/100
188/188 [=

188/188 [==============================] - 0s 378us/step - loss: 0.1282 - acc: 0.8457
Epoch 22/100
188/188 [==============================] - 0s 371us/step - loss: 0.1269 - acc: 0.8404
Epoch 23/100
188/188 [==============================] - 0s 414us/step - loss: 0.1263 - acc: 0.8457
Epoch 24/100
188/188 [==============================] - 0s 491us/step - loss: 0.1269 - acc: 0.8298
Epoch 25/100
188/188 [==============================] - 0s 446us/step - loss: 0.1198 - acc: 0.8511
Epoch 26/100
188/188 [==============================] - 0s 414us/step - loss: 0.1179 - acc: 0.8457
Epoch 27/100
188/188 [==============================] - 0s 373us/step - loss: 0.1210 - acc: 0.8351
Epoch 28/100
188/188 [==============================] - 0s 387us/step - loss: 0.1164 - acc: 0.8564
Epoch 29/100
188/188 [==============================] - 0s 385us/step - loss: 0.1120 - acc: 0.8457
Epoch 30/100
188/188 [==============================] - 0s 389us/step - loss: 0.1116 - acc: 0.8617
Epoch 31/100
188/188 [=

188/188 [==============================] - 0s 477us/step - loss: 0.2398 - acc: 0.5851
Epoch 4/100
188/188 [==============================] - 0s 361us/step - loss: 0.2348 - acc: 0.6223
Epoch 5/100
188/188 [==============================] - 0s 366us/step - loss: 0.2259 - acc: 0.6596
Epoch 6/100
188/188 [==============================] - 0s 286us/step - loss: 0.2200 - acc: 0.6809
Epoch 7/100
188/188 [==============================] - 0s 343us/step - loss: 0.2157 - acc: 0.6702
Epoch 8/100
188/188 [==============================] - 0s 382us/step - loss: 0.1952 - acc: 0.7340
Epoch 9/100
188/188 [==============================] - 0s 404us/step - loss: 0.1996 - acc: 0.7021
Epoch 10/100
188/188 [==============================] - 0s 440us/step - loss: 0.1787 - acc: 0.8191
Epoch 11/100
188/188 [==============================] - 0s 408us/step - loss: 0.1674 - acc: 0.8404
Epoch 12/100
188/188 [==============================] - 0s 420us/step - loss: 0.1590 - acc: 0.8138
Epoch 13/100
188/188 [=======

Epoch 86/100
188/188 [==============================] - 0s 424us/step - loss: 0.0378 - acc: 0.9521
Epoch 87/100
188/188 [==============================] - 0s 281us/step - loss: 0.0375 - acc: 0.9521
Epoch 88/100
188/188 [==============================] - 0s 371us/step - loss: 0.0344 - acc: 0.9734
Epoch 89/100
188/188 [==============================] - 0s 347us/step - loss: 0.0339 - acc: 0.9787
Epoch 90/100
188/188 [==============================] - 0s 339us/step - loss: 0.0322 - acc: 0.9840
Epoch 91/100
188/188 [==============================] - 0s 371us/step - loss: 0.0328 - acc: 0.9681
Epoch 92/100
188/188 [==============================] - 0s 329us/step - loss: 0.0316 - acc: 0.9734
Epoch 93/100
188/188 [==============================] - 0s 419us/step - loss: 0.0299 - acc: 0.9734
Epoch 94/100
188/188 [==============================] - 0s 334us/step - loss: 0.0265 - acc: 0.9894
Epoch 95/100
188/188 [==============================] - 0s 394us/step - loss: 0.0261 - acc: 0.9840
Epoch 96/1

- 딥러닝이 잘 구동되는 것을 확인할 수 있음
- 실행 후 10번의 테스트 값들이 출력되었음